# Chapter 8 of Partial Differential Equations: An Introduction by Walter A. Strauss

This notebook attempts to recreate the methods and examples of *Chapter 8, Computation of Solutions* from the book *Partial Differential Equations: An Introduction* by Walter A. Strauss.

The general code for a system is available in the folder `./strauss`.

In [ ]:
import strauss
import numpy as np

from IPython import display  # for rendering animations! :)

### Example 8.1.1

This first example shows the issues with a poor assumption of time step. We take the example of diffusion:

$$u_t = u_{xx}$$

with the simple step function $\phi = I_{(3,4]}$ (where $I$ is the indicator function) as our initial condition.This is approximated over 10 $x$-steps of size $\Delta x = 0.005$ each, and $\Delta t$ is chosen to be $(\Delta x)^2$.

This produces an awful oscillatory approximation (by the maximum principle the true solution should be between 0 and 1, but here it is growing past 19!). The original example did not discuss boundary conditions, but they're not relevant to what the example is demonstrating so I chose Dirichlet boundary conditions of 0 on both ends.

In [ ]:
ex_811_phi = lambda x: 1 if x==0.020 else 0
ex_811_system = strauss.Diffusion(ex_811_phi, 10, 0.005, 0.005**2)
ex_811_system.add_Dirichlet_BCs((0 ,0))

ex_811_system.print_state(0)
ex_811_system.run(5, print_state=True)

### Example 8.2.1
In this section, we solve our problem; the size of the $t$-mesh $\Delta t$ relative to the size of $\Delta x$. In particular, we create a variable

$$s = \frac{\Delta t}{(\Delta x)^2}$$

and instead of defining $\Delta x$ and $\Delta t$, we define $\Delta x$ and $s$. We find $s$ lies at the basis of a "stability criterion" for convergence of our numerical method.

Our example system is the following:

$$
u_t = u_{xx}\\
u = 0\text{ at }x \in {0, \pi}\\
u(x, 0) = \phi(x) = \begin{cases}
                    x & x \in (0, \frac{\pi}{2})\\
                    \pi - x & (\frac{\pi}{2}, \pi)
                    \end{cases}
$$

We find that the primary criterion for stability is whether $s$ is greater or smaller than $1/2$.

In [ ]:
ex_821_x_step = np.pi/20
ex_821_t_step = lambda s: s * ex_821_x_step**2

ex_821_phi = lambda x: x if x < np.pi/2 else (np.pi - x)

ex_821_stable_system = strauss.Diffusion(ex_821_phi, 20, ex_821_x_step, ex_821_t_step(5/11))
ex_821_unstable_system = strauss.Diffusion(ex_821_phi, 20, ex_821_x_step, ex_821_t_step(5/9))
ex_821_stable_system.add_Dirichlet_BCs((0 ,0))
ex_821_unstable_system.add_Dirichlet_BCs((0, 0))

In [ ]:
ex_821_stable_system.run(200)
ex_821_stable_system.graph()

In [ ]:
ex_821_unstable_system.run(200)
ex_821_unstable_system.graph()

We can also animate the two system simulations; note how you can almost 'see' the errors building up in the unstable simulation!

In [ ]:
anim = ex_821_stable_system.animate(200)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
anim = ex_821_unstable_system.animate(20)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)

### Neumann Boundary Conditions
Neumann boundary conditions can be most simply implemented as forward and backward differences, but they introduce $O(\Delta x)$ errors! To instead simply introduce $O((\Delta x)^2)$ errors we can use centred differences, treating the boundary points as 'ghost points':

$$\frac{u^n_2 - u^n_0}{2 \Delta x} = g(n), \frac{u^n_j - u^n_{j-2}}{2 \Delta x} = h(n)$$

and then solve for $u^n_0$ and $u^n_j$.

 *(NB: Strauss introduces the 'ghost points' as extra points $u^n_{-1}$ and $u^n_{j+1}$, with the centred differences centred around $u^n_j$ and $u^n_0$ instead! I chose to use the 0th and jth point for the 'ghost points' in line with using the 0th and jth point for Dirichlet boundary conditions; avoiding messing with the length of the mesh as a result.)*

We can see an example of this on the below system, identical to the previous stable example but with Neumann boundary conditions rather than Dirichlet.

In [ ]:
NBCs_system = strauss.Diffusion(ex_821_phi, 20, ex_821_x_step, ex_821_t_step(5/11))
NBCs_system.add_Neumann_BCs((0 ,0))
NBCs_system.run(200)

anim = NBCs_system.animate(200)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)